In [1]:
from IPython.core.interactiveshell import InteractiveShell

# Toujours tout afficher
InteractiveShell.ast_node_interactivity = "all"


In [1]:
import sys, os
sys.path.append(os.path.abspath(".."))

In [3]:
from scripts.inference import process_csv_file
from scripts.models import get_prediction_mistral, get_prediction_fanar
from scripts.utils import get_filename_suffix, save_mcq_file
from scripts.main import load_config,  predict_from_directory

df=predict_from_directory(config_path="../config.yaml")


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
✅ Model selected: fanar_rag
🚀 Processing file: Task1_QCM_Dev.csv
✅ توفي عن: أب، أم، أخت شقيقة واحدة.
كم مجموع أسهم التركة؟ | Fanar  | Prediction: C
✅ توفيت عن ثلاثة أشقاء وخمس شقيقات: كم عدد سهام التركة الكلي التي تقسم عليها؟ | Fanar  | Prediction: F
✅ توفي عن زوجتين وسبعة أبناء وأربع بنات: ما هو نصيب الزوجتين من التركة؟ | Fanar  | Prediction: B
✅ توفي عن: 2 ابن، 1 أخ من الأب، 2 عم ( أخ للأب من الأب )، أم، 2 بنت، 1 زوجة، 2 أخت شقيقة.
ما هو نصيب الأم بالكسر؟ | Fanar  | Prediction: D
✅ توفيت عن ابن وأخ شقيق وأختين شقيقتين. تركت قطعة حلي من الذهب كان ابنها قد اشتراها هدية لها. ما حكم هذه القطعة بعد وفاتها؟ | Fanar  | Prediction: C
✅ توفي عن ثلاثة أولاد، وبنت واحدة، وابنا ابن. ما هو النصيب النهائي لكل وارث من التركة؟ | Fanar  | Prediction: D
✅ توفي عن ابن وإخوة
ما حكم ميراث الابن المسلم من أبيه النصراني إذا مات الأب قبل إسلام الابن؟ | Fanar  | Prediction: B
✅ توفي عن: 3 ابن، 1 أب، 1 زوجة، 3 أخت شقيقة. ما هو النصيب النهائي لكل وارث من

In [2]:
from  scripts.evaluation import evaluate

# ✅ Specify your file paths
prediction_dir  =  "../results/prediction/Task1_QCM_Dev_fanar_rag_subtask1_prediction.csv"
reference_dir = "../data/Task1_QCM_Dev.csv"
output_dir = '../results/prediction/output'  # optional

# ✅ Call the evaluation function
accuracy = evaluate(reference_dir, prediction_dir, output_dir)





----------------------------------
       Evaluation Report:
----------------------------------
✅ Total examples = 1000
✅ Correct predictions = 497
✅ Accuracy = 49.7
Score file written to ../results/prediction/output/scores.json
Full prediction file copied to output directory.
